In [ ]:
%%time
%matplotlib inline
import importlib
import new_import  
importlib.reload(new_import)
from new_import import *

In [2]:
%%time
# Cấu hình Daskgateway
cluster, client = notebook_utils.initialize_dask(use_gateway=True, workers=(1,10))
# Khai báo 1 Datacube là dc
dc = datacube.Datacube()

client

An existing cluster was found. Connecting to: easihub.4d9566064818446782720d2ebc0135e1
CPU times: user 723 ms, sys: 17.8 ms, total: 741 ms
Wall time: 856 ms


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://hub.asia.easi-eo.solutions/services/dask-gateway/clusters/easihub.4d9566064818446782720d2ebc0135e1/status,


In [3]:
# cấu hình thời gian lấy ảnh và tọa độ
date_range = ('2022-09-01', '2023-10-01')
longtitude_range = (105.5, 106.4)
latitude_range = (9.2, 10.0) 

coordinates = (longtitude_range, latitude_range)

In [4]:
data_sen2 = load_data_sen2(dc, date_range, coordinates)
display(data_sen2)


No datasets require offset correction
The valid_data_mask and scale (no offset) have been applied to the reflectance bands


<xarray.Dataset>
Dimensions:      (time: 151, y: 8874, x: 9902)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-02T03:35:23.960000 ... 2023-09...
  * y            (y) float64 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06
  * x            (x) float64 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
    spatial_ref  int32 32648
Data variables:
    red          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nir          (time, y, x) float32 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    scl          (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:32648
    grid_mapping:  spatial_ref

In [6]:
data = np.load('datasets/gb_filled_cloud.npz')
fill_nan_model = joblib.load('model_train/regressor/500points/fill_nan_model_rf.pkl')

In [7]:
vh = data['vh']
vv = data['vv']
ndvi = data['ndvi']

In [11]:
ndvi.shape

(13, 8874, 9902)

In [8]:

coord_time = np.arange(13)
coord_x = data_sen2.coords['x']
coord_y = data_sen2.coords['y']


In [9]:
ndvi_xarray = xr.DataArray(ndvi, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vh_xarray = xr.DataArray(vh, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})
vv_xarray = xr.DataArray(vv, dims=["time", "y", "x"], coords={'time': coord_time, 'x': coord_x, 'y': coord_y})

In [10]:
display(ndvi_xarray)

<xarray.DataArray (time: 13, y: 8874, x: 9902)>
array([[[ 3.70842844e-01,  3.70842844e-01,  6.19868279e-01, ...,
          6.37900054e-01,  6.37840211e-01,  6.00014150e-01],
        [ 4.09747630e-01,  4.09747630e-01,  6.34832859e-01, ...,
          6.17527306e-01,  6.20078325e-01,  6.24745369e-01],
        [ 4.09747630e-01,  4.09747630e-01,  6.34832859e-01, ...,
          6.17195070e-01,  6.18910015e-01,  7.56559372e-01],
        ...,
        [ 1.72497004e-01,  4.58953112e-01,  2.66808212e-01, ...,
         -1.07296132e-01, -1.62162170e-01, -1.63636342e-01],
        [ 1.03626937e-01,  4.47405279e-01,  4.25044090e-01, ...,
         -1.84331805e-01, -1.42201796e-01, -1.60599545e-01],
        [-7.29386928e-04,  4.29869354e-01,  4.65222299e-01, ...,
         -1.11597389e-01, -1.44164741e-01, -1.15226299e-01]],

       [[-2.32006103e-01, -2.32006103e-01, -2.17974037e-01, ...,
          6.13341272e-01,  6.30607188e-01,  6.30607188e-01],
        [-2.25332618e-01, -2.25332618e-01, -2.03149900e-01, ...,
          6.80023849e-01,  6.83271587e-01,  6.83271587e-01],
        [-2.25332618e-01, -2.25332618e-01, -2.03149900e-01, ...,
          6.80023849e-01,  6.83271587e-01,  6.83271587e-01],
...
         -1.16381489e-01, -2.45650217e-01, -2.56057948e-01],
        [ 1.34953871e-01,  6.61971867e-01,  6.72875404e-01, ...,
         -1.25083700e-01, -2.16893747e-01, -1.92835450e-01],
        [ 6.93548396e-02,  5.91334820e-01,  6.62259579e-01, ...,
         -1.28072619e-01, -1.65681735e-01, -1.16094612e-01]],

       [[ 6.45346582e-01,  6.45346582e-01,  6.16828561e-01, ...,
          8.56177986e-01,  8.37543666e-01,  7.80746520e-01],
        [ 6.26065314e-01,  6.26065314e-01,  6.01952553e-01, ...,
          8.29926431e-01,  8.19785774e-01,  8.17597389e-01],
        [ 6.26065314e-01,  6.26065314e-01,  6.01952553e-01, ...,
          8.32685590e-01,  8.06613624e-01,  8.07800591e-01],
        ...,
        [ 3.88286322e-01,  6.11940384e-01,  5.17369688e-01, ...,
         -3.31825316e-02, -2.29459926e-02, -4.85436544e-02],
        [ 3.34712833e-01,  6.66666687e-01,  6.42252684e-01, ...,
         -9.60827712e-03,  2.41758358e-02,  2.19135731e-03],
        [ 2.82383412e-01,  6.62507474e-01,  6.32222235e-01, ...,
         -2.05527749e-02,  9.85917915e-03, -2.33877134e-02]]],
      dtype=float32)
Coordinates:
  * time     (time) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * x        (x) float64 5.548e+05 5.548e+05 5.548e+05 ... 6.538e+05 6.538e+05
  * y        (y) float64 1.106e+06 1.106e+06 1.106e+06 ... 1.017e+06 1.017e+06

In [11]:
label_mapping = {
    "Lua tom": "1",
    "Lua": "3",
    "CHN": "4",
    "CLN": "5",
    "TS": "6",
    "Song": "7",
    "Dat xay dung": "8",
    "Rung": "9"
}


In [12]:
train_path = "train/ST_training data_updated_1130points.shp"
# train_path = "train/updated_data/ThuanHoa_DKS_Kappa.shp"
train = load_data_geo(train_path)

In [ ]:
train

In [13]:
def extract_data_point(train, average_ndvi, dsvh, dsvv):
    loaded_datasets = {}
    for idx, point in train.iterrows():
        key = f"point_{idx + 1}"
        try:
            ndvi_data = average_ndvi.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vh_data = dsvh.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            vv_data = dsvv.sel(x=point.geometry.x, y=point.geometry.y, method='nearest').values
            loaded_datasets[key] = {
                "data": np.stack((ndvi_data, vh_data, vv_data), axis=1),
                "label": point.HT_code
            }
        except Exception as e:
            # Handle the exception if necessary
            print(f"Error at point {key}: {e}")
    return loaded_datasets

In [14]:
def create_dataset(datasets):
    data_points = []
    labels = []
    for point_key, point_data in datasets.items():
        data_array = point_data['data']
        label = point_data['label']
        for row in data_array:
            data_points.append(row)
            labels.append(label)
    return data_points, labels

In [15]:
# Extract data points and create the dataset
data_points = extract_data_point(train, ndvi_xarray, vh_xarray, vv_xarray)
X_train, y_train = create_dataset(data_points)

In [16]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [17]:
 X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [39]:
y_test

array([9, 7, 7, ..., 8, 4, 9])

In [40]:
print(X_train)

[[ 0.67069536  0.05427304  0.24030751]
 [ 0.79627126  0.05110675  0.23699677]
 [ 0.06312954  0.01186158  0.0612893 ]
 ...
 [-0.01124513  0.02648613  1.6112155 ]
 [ 0.4986132   0.02063359  0.07341269]
 [ 0.20368147  0.03412674  1.1948624 ]]


In [18]:
%%time
classifier, best_params  = cross_validate(
    train_data=(X_train, y_train), 
    model_class=classifiers['svm'], 
    param_grid=param_grids_classifier['svm'], 
    metric='accuracy',  # Use the correct scoring for MSE
    num_fold=5
)
print(best_params)

Average accuracy: 0.716176312479072
Best accuracy score: 0.7314893617021276
{'model__C': 10, 'model__kernel': 'rbf'}
CPU times: user 18.9 s, sys: 633 ms, total: 19.5 s
Wall time: 38.8 s


In [19]:
y_pred_test = classifier.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred_test)
print(f"Accuracy for test data {round(test_accuracy, 2)*100} %")

Accuracy for test data 71.0 %


In [20]:
save_model("new_classify/classify_svm.joblib", classifier)

Done!


2024-10-13 14:15:38,784 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
